# DM (device manufacturer) connects to music (device) and assign them ownership - DM

## 1. Create Invitation from agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)


IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections


## 2. Create invitation

In [2]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection d1cc52d4-7e9f-4fab-819e-3f34824e4e7a in State invitation
Connection ID d1cc52d4-7e9f-4fab-819e-3f34824e4e7a
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '1574a7ae-4fff-4bbe-b6d0-99bd2261f503', 'label': 'Alice', 'recipientKeys': ['HPzbCgs9vbNADqSan19WJXj5Lh4sojXbZRSNH4rGXuoS'], 'serviceEndpoint': 'http://192.168.65.3:8020'}
Connection Handler Called
Connection d1cc52d4-7e9f-4fab-819e-3f34824e4e7a in State request


## 4. Accept Request - Recipient first!!!!!

In [3]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection d1cc52d4-7e9f-4fab-819e-3f34824e4e7a in State response
ACCEPT REQUEST
{'their_label': 'Bob', 'invitation_mode': 'once', 'rfc23_state': 'response-sent', 'created_at': '2023-01-03 09:47:42.248896Z', 'their_did': 'NBqKhfDcnkrgJK6eRfG9Ki', 'connection_id': 'd1cc52d4-7e9f-4fab-819e-3f34824e4e7a', 'routing_state': 'none', 'invitation_key': 'HPzbCgs9vbNADqSan19WJXj5Lh4sojXbZRSNH4rGXuoS', 'accept': 'manual', 'state': 'response', 'their_role': 'invitee', 'my_did': 'D85aNxBsBeW3nACibydk7Y', 'updated_at': '2023-01-03 09:48:15.116809Z'}
state response


In [4]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
connection_id = connection["connection_id"]

Trust Ping {'thread_id': 'd4cd3154-843b-472a-8d72-af1434388bcd'}
{'their_label': 'Bob', 'invitation_mode': 'once', 'rfc23_state': 'response-sent', 'created_at': '2023-01-03 09:47:42.248896Z', 'their_did': 'NBqKhfDcnkrgJK6eRfG9Ki', 'connection_id': 'd1cc52d4-7e9f-4fab-819e-3f34824e4e7a', 'routing_state': 'none', 'invitation_key': 'HPzbCgs9vbNADqSan19WJXj5Lh4sojXbZRSNH4rGXuoS', 'accept': 'manual', 'state': 'response', 'their_role': 'invitee', 'my_did': 'D85aNxBsBeW3nACibydk7Y', 'updated_at': '2023-01-03 09:48:15.116809Z'}
Is Active? response
Connection Handler Called
Connection d1cc52d4-7e9f-4fab-819e-3f34824e4e7a in State active


In [5]:
response = await agent_controller.terminate()
print(response)

None


# Give MUSIC the VC of ownership and origin - DM

## 1. Write Ownership schema

In [31]:
# Define you schema name - must be unique on the ledger
schema_name = "music_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["origin", "name", "time", "version", "author", "owner"]

# Write schema to ledger
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

# Write cred. def.
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)


# # Update schema
# attributes = ["author", "name", "time", "is_original", "type"]
# schema_version = "0.0.2"
# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# updated_schema_id = response["schema_id"]
# print(updated_schema_id)

# # Write another Credential Definition for the Updated Schema
# response = await agent_controller.definitions.write_cred_def(updated_schema_id, support_revocation=True)
# updated_cred_def_id = response["credential_definition_id"]
# print(updated_cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1
PQRXDxdGqQGSZ8z69p4xZP:3:CL:18:default


## 2. Write VC

In [32]:
nym = await agent_controller.wallet.get_public_did()
print(nym["result"]["did"])
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did='PQRXDxdGqQGSZ8z69p4xZP')
print(created_schema)
schema_id = created_schema["schema_ids"][-1]

PQRXDxdGqQGSZ8z69p4xZP
{'schema_ids': ['PQRXDxdGqQGSZ8z69p4xZP:2:test_revocable_schema:0.0.1', 'PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1']}


In [33]:
definitions = await agent_controller.definitions.search_created(schema_id=schema_id)
print(definitions["credential_definition_ids"])
cred_def_id = definitions["credential_definition_ids"][-1]

['PQRXDxdGqQGSZ8z69p4xZP:3:CL:18:default']


In [34]:
response = await agent_controller.connections.get_connections()
results = response['results']
print(len(results))
# print("Results : ", results)
if len(results) > 0:
    connection = response['results']
    for con in connection:
        print("Connection :", con)
        if con['state'] == 'active':       
            connection_id = con["connection_id"]
            print("Active Connection ID : ", connection_id)
        else: print("Inactive Connection ID: ", con["connection_id"], "with state: ", con["state"])
else:
    print("You must create a connection")

5
Connection : {'invitation_mode': 'once', 'state': 'active', 'accept': 'manual', 'invitation_key': 'HP7gvAJMddRagAmzzTXK9s9a5yQLArvrowgaezhy3pgT', 'their_did': '2DmECYnbF1ShhKhRr7WJTq', 'their_role': 'invitee', 'connection_id': 'b3f010a2-4dcc-4887-b44f-b68748a87751', 'rfc23_state': 'completed', 'their_label': 'Bob', 'my_did': 'EqGVJqFX6psNNnhjLsEGck', 'updated_at': '2023-01-03 05:59:43.798364Z', 'routing_state': 'none', 'created_at': '2023-01-03 05:58:58.974222Z'}
Active Connection ID :  b3f010a2-4dcc-4887-b44f-b68748a87751
Connection : {'invitation_mode': 'once', 'state': 'active', 'accept': 'manual', 'invitation_key': '8yDBAPXWfTEKdKCdRHBr7rKVpJ5pZkQ2bLGoJEXFbxCX', 'their_did': 'YTZXU7cCnfKDw3cCqUhDbA', 'their_role': 'invitee', 'connection_id': 'ff6ac676-00e0-4b80-a588-6b71f6d7c02e', 'rfc23_state': 'completed', 'their_label': 'Bob', 'my_did': 'LuvDTHNQGRpNWPk79mbEU2', 'updated_at': '2023-01-03 06:12:35.766848Z', 'routing_state': 'none', 'created_at': '2023-01-03 06:12:04.524296Z'}
A

In [35]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "author", "value": nym["result"]["did"]},
    {"name": "name", "value": "balabababa music"},
    {"name": "time", "value": time},
    {"name": "origin", "value": nym["result"]["did"]},
    {"name": "owner", "value": nym["result"]["did"]},
    {"name": "version", "value": "v1"}
]
print(credential_attributes)


# send credential
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

[{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'name', 'value': 'balabababa music'}, {'name': 'time', 'value': '01-03-2023, 09:38:38'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'version', 'value': 'v1'}]
Credential exchange b2eec7f3-04b5-4a89-97a3-437f48eb2ff8, role: issuer, state: offer_sent


## 4. Wait for credential exchange

In [23]:
response = await agent_controller.issuer.get_records()
print(response['results'])
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

[{'revoc_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default:CL_ACCUM:b3dcbef1-f919-4a66-b1e9-f8141e93d801', 'credential_offer_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/offer-credential', '@id': '12f87972-4fe1-4cc9-aadb-e55aab97df5b', '~thread': {}, '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'comment': 'create automated credential exchange', 'offers~attach': [{'@id': 'libindy-cred-offer-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJzY2hlbWFfaWQiOiAiUFFSWER4ZEdxUUdTWjh6NjlwNHhaUDoyOnRlc3RfcmV2b2NhYmxlX3NjaGVtYTowLjAuMSIsICJjcmVkX2RlZl9pZCI6ICJQUVJYRHhkR3FRR1NaOHo2OXA0eFpQOjM6Q0w6MTI6ZGVmYXVsdCIsICJrZXlfY29ycmVjdG5lc3NfcHJvb2YiOiB7ImMiOiAiNjI5NzUwMjAxNDgwMTcwOTQwNDcwMDc0NjE5OTAyNTA5NDY2ODM0OTU1Njk1MzU4ODIyNDk0MzQ1ODQyNTU3OTgxNjI2ODcyNDI5ODciLCAieHpfY2FwIjogIjEwMDcyMzY5NDc3MjA5ODQ2OTMwNDU2NzYwNzg4OTU1MDk3NDA1Mjg2MTI1MjQ2ODIzMjQ3NjcwNTMyNDI4NjA5NjMxNTIxOTQwOTA5MjMxNTI4MDg2MTY4MTY3MjM5ODcwMjE5OD

In [24]:
## Remove records
response = await agent_controller.issuer.remove_record(record_id)
print(response)

{}


In [25]:
response = await agent_controller.terminate()
print(response)

None
